In [1]:
# imports(default)
import sys
from glob import glob
from tqdm import tqdm
import numpy as np
import gensim
import MeCab
sys.path.append("../src")
from models import *
from utils import *
import chainer
from prefetch_generator import BackgroundGenerator
from IPython.display import clear_output
from matplotlib import pylab as plt

/home/shido/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
char_files = glob("../dataset/charactor/*.txt")
conv_files = glob("../dataset/conversation/*.txt")

In [3]:
char_texts = [read_file(file) for file in char_files]
conv_texts = [read_file(file) for file in conv_files]

In [4]:
parser = Parser()

In [5]:
vocab = set()
for conv in conv_texts:
    for line in conv:
        vocab = vocab.union(set(parser.parse(line)))
for conv in char_texts:
    for line in conv:
        vocab = vocab.union(set(parser.parse(line)))

In [6]:
word2id = {}
id2word = {}

for e, word in enumerate(["<PAD>", "<BOS>", "<EOS>", "<UNK>"] + sorted(list(vocab))):
    word2id[word] = e
    id2word[e] = word

In [7]:
tokenizer = Tokenizer(word2id, id2word)
char_seqs = [[tokenizer.encode(parser.parse(line)) for line in text] for text in char_texts]
conv_seqs = [[tokenizer.encode(parser.parse(line)) for line in text] for text in conv_texts]

In [8]:
conv_x = []
conv_y = []

In [9]:
for text in conv_seqs:
    conv_x += text[0::2]
    conv_y += text[1::2]

In [10]:
EMBEDDING_SIZE = 200
NUM_UNITS = 400
SEQ_LEN = 150
LAYERS = 2
BATCH_SIZE = 256
VOCAB = len(word2id)

In [11]:
# setting model
model = Model(VOCAB, SEQ_LEN, EMBEDDING_SIZE, NUM_UNITS, LAYERS)
model.to_gpu(0)
epochs = 100
batch_size = 128
optimizer = chainer.optimizers.Adam(1e-3)
optimizer.setup(model)
history = {"loss":[]}

In [12]:
# Setting Data Generator
def mygen(X, Y, batch_size, train=True):
    if train:
        newindex = list(np.random.permutation(len(X)))
        X = [X[i] for i in newindex]
        Y = [Y[i] for i in newindex]
    for i in range(0, len(X), batch_size):
        x = [np.array(x, "int32") for x in X[i:i+batch_size]]
        y = [np.array(y, "int32") for y in Y[i:i+batch_size]]
        yield(x, y)

In [13]:
# training
for epoch in range(epochs):
    
    # train
    tmp_loss = 0.0
    gen = BackgroundGenerator(mygen(conv_x, conv_y, batch_size))
    for x, y in tqdm(gen, "epoch-{:02d} train".format(epoch + 1)):
        x = to_device0(x)
        y = to_device0(y)
        x = F.pad_sequence(x, SEQ_LEN)
        y = F.pad_sequence(y, SEQ_LEN)
        
        loss = model.get_loss(x, y)
        model.cleargrads()
        loss.backward()
        optimizer.update()
        
        tmp_loss += float(loss.data) * len(y)
    
    tmp_loss /= len(conv_x)
    history["loss"].append(tmp_loss)
    print(tmp_loss)

    # checkpoint
    if tmp_loss == min(history["loss"]):
        model.to_cpu()
        chainer.serializers.save_npz("../models/seq2seq.npz", model)
        model.to_gpu()

epoch-01 train: 25it [00:09,  2.67it/s]


284.1356550331721


epoch-02 train: 25it [00:08,  3.00it/s]


101.66193209693546


epoch-03 train: 25it [00:08,  2.99it/s]


...

21.59085955180819


epoch-99 train: 25it [00:08,  2.93it/s]


21.346767252361964


epoch-100 train: 25it [00:08,  2.95it/s]


20.954856104775082


In [13]:
model.training_model.load_weights("../models/seq2seq.npz")

In [14]:
i = 111
source = conv_x[i:i+2]
inputs = pad_sequences(source, 150, padding="post", truncating="post")
states = model.predict_sequence(inputs, 50, mode="random", alpha=1)
print(tokenizer.decode_batch(source))
print(tokenizer.decode_batch(states))

['どんなジャンルの音楽が好きですか？', 'どんなジャンルが好きなんですか？']
['海に行かないんですか？', 'すごく簡単です。']


In [20]:
query = "明日は何をしますか"
print(parser.parse(query))
query = [tokenizer.encode(parser.parse(query))]
print(tokenizer.decode(query))
query = pad_sequences(query, 150, padding="post", truncating="post")
result = model.predict_sequence(query, 50, mode="random", alpha=.8)
print(tokenizer.decode(result[0]))

['明日', 'は', '何', 'を', 'し', 'ます', 'か']

ぼくはペットへ大好きです
